PyTorch는 코드 내의 다양한 Pytorch 연산에 대한 시간과 메모리 비용을 파악하는 데 유용한 프로파일러(profiler) API를 포함하고 있습니다. 프로파일러는 코드에 쉽게 통합될 수 있으며, 프로파일링 결과는 표로 출력되거나 JSON 형식의 추적(trace) 파일로 반환될 수 있습니다.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import torch
from torch import nn
import torch.autograd.profiler as profiler

### 프로파일러를 이용하여 성능 디버깅하기
프로파일러는 모델에서 성능의 병목을 파악할 때 유용할 수 있습니다. 이번 예제에서, 두 가지 하위 작업을 수행하는 사용자 정의 모듈을 만들겠습니다:

* 입력에 대한 선형 변환

* 변환 결과를 이용한 마스크 텐서(mask Tensor)에서 인덱스 추출

각 하위 작업들에 대한 코드는 profiler.record_function("label") 을 이용하여 레이블된 컨텍스트 매니저(context manager) 들에 의해 감쌉니다. 프로파일러의 출력에서, 하위 작업들의 모든 연산에 대한 집계(aggregate) 성능 지표들이 해당 레이블 아래 나타나게 됩니다.

프로파일러를 사용하는 것은 약간의 오버헤드가 발생하며, 코드를 분석할 때에만 사용하는 것이 가장 좋습니다. 만일 실행시간을 벤치마킹하는 경우에는 이를 제거하는 것을 잊지 마십시오.

In [3]:
class MyModule(nn.Module):
    def __init__(self, in_features: int, out_features: int, bias: bool = True):
        super(MyModule, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias)

    def forward(self, input, mask):
        with profiler.record_function("LINEAR PASS"):
            out = self.linear(input)

        with profiler.record_function("MASK INDICES"):
            threshold = out.sum(axis=1).mean().item()
            hi_idx = np.argwhere(mask.cpu().numpy() > threshold)
            hi_idx = torch.from_numpy(hi_idx).cuda()

        return out, hi_idx


### 순전파 단계(forward pass) 프로파일링하기
입력과 마스크 텐서, 그리고 모델을 임의로 초기화합니다.

프로파일러를 실행하기 전, 정확한 성능 벤치마킹을 보장하기 위해 CUDA를 워밍업(warm-up) 시킵니다. 모델의 순전파 단계를 profiler.profile 컨텍스트 매니저를 통해 감쌉니다. with_stack=True 인자는 연산의 추적(trace) 파일 내부에 파일과 줄번호를 덧붙입니다.